In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
import os
rc('text', usetex=True)
rc('font', family='serif')
% matplotlib inline

In [ ]:
ab = str(raw_input("Model: "))
model = ab + "_millennium"
path = "/home/esteban/Escritorio/Practica/Data/" + model
os.chdir( path )

In [ ]:
halos_data = np.load('halos_masscut_2.0.npy')

fofid = halos_data[:,0]         # Halo's fofid
logmass = halos_data[:,1]       # Halo's log mass
All = halos_data[:,2]           # Num of galaxies in the halo
Cent = halos_data[:,3]          # Num of Cent //
Sats = halos_data[:,4]          # Num of Sats //
x = halos_data[:,5]             # Halo x-position 

In [ ]:
hod_data= np.load('hod_masscut_2.0.npy')
bins = hod_data[:,0]
HOD = hod_data[:,1]
Cent_plot = hod_data[:,2]
Sats_plot = hod_data[:,3]
Dmin, Dmax = 10,15.
NBIN = 30
bin_width = (Dmax-Dmin)/NBIN
bins_left = bins - bin_width *.5
# bins[15], HOD[15] ############### punto a analizar

In [ ]:
bins_left

In [ ]:
mask1 = (logmass > bins_left[14]) & (logmass < bins_left[15])
mask2 = (logmass > bins_left[-3]) & (logmass < bins_left[-2])

In [ ]:
# eliminar los ceros es consistente, solo estoy analizando halos que cumplen con el criterio de masa (hacerlo tambien para sfr)
All_mask1, All_mask2 = All[mask1], All[mask2]
#Cent_mask1, Cent_mask2 = Cent[mask1], Cent[mask2]
Sats_mask1, Sats_mask2 = Sats[mask1], Sats[mask2]

mask_zeros1, mask_zeros2 = All_mask1 == 0, All_mask2 == 0
All_mask1, All_mask2 = All_mask1[~mask_zeros], All_mask2[~mask_zeros2]
#Cent_mask1, Cent_mask2 = Cent_mask1[~mask_zeros1], Cent_mask2[~mask_zeros2]
Sats_mask1, Sats_mask2 = Sats_mask1[~mask_zeros1], Sats_mask2[~mask_zeros2]   

In [ ]:
All_mask2, Sats_mask2, mask_zeros2

In [ ]:
f, ax = plt.subplots(nrows=2, ncols=2, figsize = [10,10])

ax[0,0].hist(All_mask1, bins = np.arange(10), alpha=.3, facecolor = 'b')
#ax[0,0].set_title("All-Galaxies Histogram")
ax[0,1].hist(Sats_mask1, bins = np.arange(10), alpha = .3, facecolor = 'b')
#ax[0,1].set_title("Sats-Galaxies Histogram")
ax[1,0].hist(All_mask2, bins = np.arange(100), alpha = .3, facecolor = 'r')
#ax[0,1].set_title("Sats-Galaxies Histogram")
ax[1,1].hist(Sats_mask2, bins = np.arange(100), alpha = .3, facecolor = 'r')
#ax[0,1].set_title("Sats-Galaxies Histogram")

for row in ax:
    for col in row:
        col.set_xlabel(r"\tt{Number of galaxies}", fontsize = 12)
        col.set_ylabel(r"\tt{N}", fontsize = 12)
        col.set_yscale('log')

In [ ]:
plt.plot(bins, HOD, 'k.')
plt.plot(bins, Cent_plot, 'r-')
plt.plot(bins, Sats_plot, 'b-')
plt.axvline(bins_left[14], linestyle = '--', color = 'black')
plt.axvline(bins_left[15], linestyle = '--', color = 'black')
plt.axvline(bins_left[-3], linestyle = '--', color = 'black')
plt.axvline(bins_left[-2], linestyle = '--', color = 'black')
plt.ylim(-2, 3)

In [ ]:
type_gal = np.load('type.npy')
log_centralmvir = np.log10(np.load('mdhalo.npy') + 1e-10) + 10

In [ ]:
logm_min = 10
logm_max = 15.
NBIN = 30

mass_labels = ((log_centralmvir - logm_min)/(logm_max - logm_min) * NBIN).astype(int)
bin_width = (logm_max - logm_min)/NBIN
bins = np.array([logm_min + bin_width*i for i in range(NBIN)]) + bin_width *.5

In [ ]:
mask = (mass_labels >= NBIN) | (mass_labels < 0)
type_gal = type_gal[~mask]
log_centralmvir = log_centralmvir[~mask]
mass_labels = mass_labels[~mask]

In [ ]:
%%time
Ngal = np.zeros(NBIN)
Ncen = np.zeros(NBIN)
Nsat = np.zeros(NBIN)

for i in range(NBIN):
    
    idx = np.where(mass_labels == i)[0]
    Ngal[i] = len(idx)
    Ncen[i] = len(np.where(type_gal[idx] == 0)[0])
    Nsat[i] = len(np.where(type_gal[idx] != 0)[0])

In [ ]:
#path = "/home/esteban/Escritorio/Practica/Plots/G13_millennium" # Guo2013    (MPA)
#path = "/home/esteban/Escritorio/Practica/Plots/G14_millennium"  # Gonzalez2014  (Durham)
path = "/home/esteban/Escritorio/Practica/Plots/L16_millennium" # Lacey2016     (Durham)
os.chdir( path )

In [ ]:
bins_hist = np.linspace(logm_min, logm_max, NBIN, endpoint=False)
f, ax = plt.subplots(ncols=2, figsize = [12,5])

ax[0].bar(bins_hist, np.log10(Ngal + 1e-10), bin_width, lw = .1, facecolor='green', alpha=.3, label = 'All')
ax[0].plot(bins, np.log10(Ncen + 1e-10), 'r--', label = 'Cent')
ax[0].plot(bins, np.log10(Nsat + 1e-10), 'b--', label = 'Sats')
ax[0].set_ylim(0,10)
ax[0].set_ylabel(r"$log(N)$", fontsize=15)
ax[1].bar(bins_hist, Ngal, bin_width, lw = .1, facecolor='green', alpha=.3, label = 'All')
ax[1].plot(bins, Ncen, 'r--', label = 'Cent')
ax[1].plot(bins, Nsat, 'b--', label = 'Sats')
ax[1].ticklabel_format(style='sci', axis='y', scilimits=(0,0))
ax[1].set_ylabel('N', fontsize=15)

for a in ax:
    a.set_xlim(logm_min, logm_max)
    a.set_xlabel(r"$log(M_h / h^{-1}M_{\odot})$", fontsize=15)
    a.legend(loc='upper right', frameon = False, prop={'size':10})
    a.minorticks_on()
    
f.tight_layout()
f.savefig('histograms.pdf')